In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
from cellpose import io
from liams_funcs import *
from features import *
from pytorch import *

import pandas as pd
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
control_dir = '../control_images/'
penetramax_dir = '../penetramax_images/'

control_fnames = [control_dir + fname for fname in os.listdir(control_dir)]
penetramax_fnames = [penetramax_dir + fname for fname in os.listdir(penetramax_dir)]

In [4]:
#ds = Dataset(control_fnames, penetramax_fnames)
#_ = ds.createImageDataset(final_size=224)

cell_data = LoadImageDataset('pytorch_dataset/cells/')
nucleus_data = LoadImageDataset('pytorch_dataset/nuclei/')

In [15]:
from torch.utils.data import DataLoader, random_split

train_data, val_data = random_split(nucleus_data, [0.8, 0.2])
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

In [7]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

n_epochs = 10
clf = Resnet18(train_layers=2).to(device)
loss_fn = CrossEntropyLoss()
opt = Adam(clf.parameters(), lr=1e-3)

train_losses = [get_loss(clf, train_dataloader, loss_fn)]
val_losses = [get_loss(clf, val_dataloader, loss_fn)]

for _ in range(n_epochs):
    for X, y in tqdm(train_dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = clf(X)
        loss = loss_fn(y_pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

    train_losses.append(get_loss(clf, train_dataloader, loss_fn))
    val_losses.append(get_loss(clf, val_dataloader, loss_fn))

  0%|          | 0/242 [00:00<?, ?it/s]

100%|██████████| 242/242 [36:10<00:00,  8.97s/it]
